In [1]:
import numpy as np
import os
import pickle
import h5py
#import pandas as pd

from __future__ import print_function
import keras
from keras.datasets import cifar100
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D


Using TensorFlow backend.


In [2]:
# get original raw data
def get_data(raw):
    raw_float = np.array(raw, dtype=float) 
    images = raw_float.reshape([-1, 3, 32, 32])
    images = images.transpose([0, 2, 3, 1])
    return images

# get labels
def get_labels(labels):
    return np.array(labels)

# input local data
def unpickle(file):
    import pickle
    fo = open(file, 'rb')
    dict = pickle.load(fo, encoding='iso-8859-1')
    fo.close()
    return dict

# preprocessing - normalize
def normalize(train, test):
    mean, std = train.mean(), test.std()
    train = (train - mean) / std
    test = (test - mean) / std
    return train, test

In [3]:
x_train = get_data(unpickle('train')['data'])
#print(x_train.shape)
y_train = get_labels(unpickle('train')['fine_labels'])
y_train = y_train.transpose()
y_train.shape = (50000,1)
#print(y_train.shape)

x_test = get_data(unpickle('test')['data'])
#print(x_test.shape)
y_test = get_labels(unpickle('test')['fine_labels'])
y_test = y_test.transpose()
y_test.shape = (10000,1)
#print(y_test.shape)

In [4]:
classes = 100

y_train = keras.utils.to_categorical(y_train, classes)
y_test = keras.utils.to_categorical(y_test, classes)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#x_train /= 255.0
#x_test /= 255.0

In [5]:
x_train, x_test = normalize(x_train, x_test)

In [6]:
# build training model

model = Sequential()
# layer one
model.add(Conv2D(128,(3,3), padding = 'same', input_shape = x_train.shape[1:]))
model.add(Activation('elu'))
model.add(Conv2D(128, (3,3)))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# layer two
model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Activation('elu'))
model.add(Conv2D(256, (3, 3)))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# layer three
model.add(Conv2D(512, (3, 3), padding='same'))
model.add(Activation('elu'))
model.add(Conv2D(512, (3, 3)))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('elu'))
model.add(Dropout(0.5))
model.add(Dense(classes))
model.add(Activation("softmax"))


opt = keras.optimizers.rmsprop(lr=0.0001, decay = 1e-6)

model.compile(loss = 'categorical_crossentropy', optimizer=opt, metrics = ['accuracy'])

In [7]:
epochs = 50
data_augmentation = False
predictions = 20
batch_size = 64
validation = []

# train the model and record loss and accuracy
print("**********begin***************")
hist = model.fit(x_train, y_train, batch_size= batch_size, epochs = epochs, validation_data = (x_test, y_test), shuffle = True)
with open('best1.txt','w') as f:
    f.write(str(hist.history))
print("***********done*************")

**********begin***************
Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 150s 3ms/step - loss: 3.8380 - acc: 0.1215 - val_loss: 3.5437 - val_acc: 0.1738
Epoch 2/50
50000/50000 [==============================] - 139s 3ms/step - loss: 3.1546 - acc: 0.2402 - val_loss: 2.8100 - val_acc: 0.3083
Epoch 3/50
50000/50000 [==============================] - 139s 3ms/step - loss: 2.8289 - acc: 0.3041 - val_loss: 2.5867 - val_acc: 0.3581
Epoch 4/50
50000/50000 [==============================] - 139s 3ms/step - loss: 2.5940 - acc: 0.3540 - val_loss: 2.4016 - val_acc: 0.3900
Epoch 5/50
50000/50000 [==============================] - 139s 3ms/step - loss: 2.4162 - acc: 0.3902 - val_loss: 2.2048 - val_acc: 0.4366
Epoch 6/50
50000/50000 [==============================] - 139s 3ms/step - loss: 2.2605 - acc: 0.4234 - val_loss: 2.1607 - val_acc: 0.4435
Epoch 7/50
50000/50000 [==============================] - 139s 3ms/step - loss: 2.1196 - ac

In [8]:
# evaluate the accuracy of model

score = model.evaluate(x_test, y_test, verbose = 0)

In [9]:
print('Test loss:', score[0])

Test loss: 2.90024401932


In [10]:
print('Test accuracy:', score[1])

Test accuracy: 0.5653


In [12]:
#from sklearn.metrics import recall_score

#y_true = [0, 1, 2, 0, 1, 2]
#y_pred = [0, 2, 1, 0, 0, 1]
#recall_score(y_true, y_pred, average='macro')


In [13]:
#from sklearn.metrics import precision_score

#y_true = [0, 1, 2, 0, 1, 2]
#y_pred = [0, 2, 1, 0, 0, 1]
#precision_score(y_true, y_pred, average='macro')

In [15]:
#import seaborn as sns
#from sklearn.metrics import confusion_matrix
#import matplotlib.pyplot as plt

#y_pred = model.predict_classes(x_test)
#print(y_pred)
#y_test = get_labels(unpickle('test')['fine_labels'])
#print(y_test)

In [16]:
# get the confusion matrix 
#cm = confusion_matrix(y_test, y_pred)

#print("Confusion Matrix")
#print(cm.shape)
#plt.figure(figsize=(10,10))
#sns.heatmap(cm[45:65,45:65],annot=True,fmt="d",cmap="YlGnBu")
#plt.ylabel("true labels")
#plt.xlabel("predict labels")
#plt.title("Confusion Matrix")